# Keras函数式API

## 与Squential模型的比较
例子中将**Model**对象实例化只需要一个输入张量和一个输出张量。Keras会在后台检索从`input_tensor`到`output_tensor`所包含的每一层，并将这些层组合成一个类图的数据结构，即一个Model。当然，这种方法有效的原因在于，`output_tensor`是通过对`input_tensor`进行多次变换得到的。如果试图利用不相关的输
入和输出来构建一个模型，那么会得到`RuntimeError`

In [1]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# 定义Squential模型
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# 对应的函数式API
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)
# Model 类将输入张量和输出张量转换为一个模型
model = Model(input_tensor, output_tensor)

print(model.summary())
print(seq_model.summary())

Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                2080      
______________________________

In [2]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

import numpy as np

x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=1, batch_size=128)

score = model.evaluate(x_train, y_train)
print(score)

Epoch 1/1
1000/1000 [==============================] - 0s 27us/step
12.171489685058594


## 多输入模型
函数式API可用于构建具有多个输入的模型。通常情况下，这种模型会在某一时刻用一个可以组合多个张量的层将不同的输入分支合并，张量组合方式可能是相加、连接等。这通常利用Keras的合并运算来实现，比如`keras.layers.add`、`keras.layers.concatenate`等。

来看一个非常简单的多输入模型示例——一个问答模型。典型的问答模型有两个输入：一个自然语言描述的问题和一个文本片段（比如新闻文章），后者提供用于回答问题的信息。然后模型要生成一个回答，在最简单的情况下，这个回答只包含一个词，可以通过对某个预定义的词表做softmax得到:

<img src="./images/问答模型.png" style="zoom:50%" />

In [3]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# 文本输入是一个长度可变的整数序列。注意，可以选择对输入进行命名
text_input = Input(shape=(None,), dtype='int32', name='text')
# 嵌入长度为64的向量
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
# 利用LSTM将向量编码为单个向量
encoded_text = layers.LSTM(32)(embedded_text)

# 对问题文本进行相同的处理
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# 将编码后的问题和文本连接起来，并上面添加一个softmax分类器
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [4]:
import numpy as np
import keras

num_samples = 1000
max_length = 100
text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
# 回答是one-hot编码
answers = keras.utils.to_categorical(answers, answer_vocabulary_size)

# 训练方法1
model.fit([text, question], answers, epochs=1, batch_size=128)
# 训练方法2
model.fit({'text': text, 'question': question}, answers, epochs=1, batch_size=128)

/Users/s33c0f1/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 6.2146 - acc: 0.0010
Epoch 1/1
1000/1000 [==============================] - 0s 487us/step - loss: 6.1963 - acc: 0.0310


## 多输出模型
同多输入模型相同，函数式API还可以构建多输出模型。重要的是，训练这种模型需要能够对网络的各个头指定不同的损失函数，例如，年龄预测是标量回归任务，而性别预测是二分类任务，二者需要不同的训练过程。但是，梯度下降要求将一个标量最小化，所以为了能够训练模型，必须将这些损失合并为单个标量。合并不同损失最简单的方法就是对所有损失求和。在Keras中，可以在编译时使用损失组成的列表或字典来为不同输出指定不同损失，然后将得到的损失值相加得到一个全局损失，并在训练过程中将这个损失最小化。

<img src="./images/多输出模型.png" style="zoom:50%" />

**Note**:严重不平衡的损失贡献会导致模型表示针对单个损失值最大的任务优先进行优化，而不考虑其他任务的优化。为了解决这个问题，可以为每个损失值对最终损失的贡献分配不同大小的重要性。如果不同的损失值具有不同的取值范围，那么这一方法尤其有用。比如，用于年龄回归任务的均方误差（MSE）损失值通常在3~5 左右，而用于性别分类任务的交叉熵损失值可能低至0.1。在这种情况下，为了平衡不同损失的贡献，可以让交叉熵损失的权重取10，而MSE 损失的权重取0.5。

In [5]:
from keras import layers
from keras import Input
from keras.models import Model


vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# 输出层
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [ ]:
model.compile(optimizer='rmsprop', 
              loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25, 'income': 1., 'gender': 10.})

model.fit(posts, {'age': age_targets, 'income': income_targets, 'gender': gender_targets}, epochs=10, batch_size=64)

## 有向无环图
利用函数式API，不仅可以构建多输入和多输出的模型，而且还可以实现具有复杂的内部拓扑结构的网络。
Keras 中的神经网络可以是层组成的任意**有向无环图（directed acyclicgraph）**。**无环（acyclic）** 这个限定词很重要，即这些图不能有循环。张量**x**不能成为生成**x**的某一层的输入。唯一允许的处理循环（即循环连接）是循环层的内部循环。
### Inception 模块
Inception 是一种流行的卷积神经网络的架构类型, 由Google的Christian Szegedy及其同事在2013—2014年研发。
> a LIN M, CHEN Q, YAN S. Network in network [C]//International Conference on Learning Representations, 2014.

示例图为Inception V3

<img src="./images/Inception模块.png" style="zoom:40%" />

**1×1 卷积的作用**:
卷积能够在输入张量的每一个方块周围提取空间图块，并对所有图块应用相同的变换。极端情况是提取的图块只包含一个方块。这时卷积运算等价于让每个方块
向量经过一个Dense 层：它计算得到的特征能够将输入张量通道中的信息混合在一起，但不会将跨空间的信息混合在一起（因为它一次只查看一个方块）。这种1×1 卷积［也叫作**逐点卷积（pointwise convolution）**］是Inception 模块的特色，它有助于区分开通道特征学习和空间特征学习。如果你假设每个通道在跨越空间时是高度自相关的，但不同的通道之间可能并不高度相关，那么这种做法是很合理的。

完整的Inception V3架构内置于**Keras**中，位置在`keras.applications.inception_v3.InceptionV3`，其中包括在ImageNet 数据集上预训练得到的权重。与其密切相关的另一个模型是Xception，它也是**Keras**的applications模块的一部分。

In [ ]:
from keras import layers

# 该示例假设输入x为四维张量，且每个分支都有相同的步幅值（2），用于保持所有分支输出具有相同的尺寸

branch_a = layers.Conv2D(128, 1,activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

### 残差连接(ResNet)
残差连接（residual connection）是一种常见的类图网络组件。2015年末，来自微软的何恺明等人在ILSVRC ImageNet 挑战赛中获胜，其中引入了这一方法。残差连接解决了困扰所有大规模深度学习模型的两个共性问题：**梯度消失**和**表示瓶颈**。
> HE K, ZHANG X, REN S, et al. Deep residual learning for image recognition [C]//Conference on Computer Vision and
Pattern Recognition, 2016.

残差连接是让前面某层的输出作为后面某层的输入，从而在序列网络中有效地创造了一条捷径。前面层的输出没有与后面层的激活连接在一起，而是与后面层的激活相加（这里假设两个激活的形状相同）。如果它们的形状不同，可以用一个线性变换将前面层的激活改变成目标形状（例如，这个线性变换可以是不带激活的`Dense 层`；对于卷积特征图，可以是不带激活`1×1卷积`）。

---
* **深度学习中的表示瓶颈**:
在Sequential 模型中，每个连续的表示层都构建于前一层之上，这意味着它只能访问前一层激活中包含的信息。如果某一层太小（比如特征维度太低），那么模型将会受限于该层激活中能够塞入多少信息。你可以通过类比信号处理来理解这个概念：假设你有一条包含一系列操作的音频处理流水线，每个操作的输入都是前一个操作的输出，如果某个操作将信号裁剪到低频范围（比如0~15 kHz），那么下游操作将永远无法恢复那些被丢弃的频段。任何信息的丢失都是永久性的。残差连接可以将较早的信息重新注入到下游数据中，从而部分解决了深度学习模型的这一问题。

* **深度学习中的梯度消失**:
反向传播是用于训练深度神经网络的主要算法，其工作原理是将来自输出损失的反馈信号向下传播到更底部的层。如果这个反馈信号的传播需要经过很多层，那么信号可能会变得非常微弱，甚至完全丢失，导致网络无法训练。这个问题被称为梯度消失（vanishing gradient）。深度网络中存在这个问题，在很长序列上的循环网络也存在这个问题。在这两种情况下，反馈信号的传播都必须通过一长串操作。我们已经知道LSTM层是如何在循环网络中解决这个问题的：它引入了一个携带轨道（carry track），可以在与主处理轨道平行的轨道上传播信息。残差连接在前馈深度网络中的工作原理与此类似，但它更加简单：它引入了一个纯线性的信息携带轨道，与主要的层堆叠方向平行，从而有助于跨越任意深度的层来传播梯度。

In [ ]:
from keras import layers

# 如果特征图的尺寸相同，在Keras中使用是恒等残差连接（identityresidual connection）
# 假设x为四维张量
x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
# 将原始x 与输出特征相加
y = layers.add([y, x])


# 如果特征图的尺寸不同，则使用线性残差连接（linear residualconnection）
from keras import layers
x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

# 使用1×1卷积，将原始x张量线性下采样为与y具有相同的形状
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)
# 将残差张量与输出特征相加
y = layers.add([y, residual])

### 共享层权重
函数式API 还有一个重要特性，那就是能够多次重复使用一个层实例。如果对一个层实例调用两次，而不是每次调用都实例化一个新层，那么每次调用可以重复使用相同的权重。这
样可以构建具有共享分支的模型，即几个分支全都共享相同的知识并执行相同的运算。也就是说，这些分支共享相同的表示，并同时对不同的输入集合学习这些表示。

举个例子，假设一个模型想要评估两个句子之间的语义相似度。这个模型有两个输入（需要比较的两个句子），并输出一个范围在0~1 的分数，0 表示两个句子毫不相关，1 表示两个句子完全相同或只是换一种表述。这种模型在许多应用中都很有用，其中包括在对话系统中删除重复的自然语言查询。

在这种设置下，两个输入句子是可以互换的，因为语义相似度是一种对称关系，A 相对于B 的相似度等于B 相对于A 的相似度。因此，需要用一个LSTM层来处理两个句子。这个LSTM 层的表示（即它的权重）是同时基于两个输入来学习的。将其称为**连体LSTM(Siamese LSTM)** 或 **共享LSTM(shared LSTM)** 模型。

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

# 将一个LSTM 层实例化一次
lstm = layers.LSTM(32)

left_input = Input(shape=(None, 128))
left_output = lstm(left_input)
right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

# 将模型实例化并训练：训练这种模型时，基于两个输入对LSTM层的权重进行更新
model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)

### 将模型作为层
在函数式API 中，可以像使用层一样使用模型。实际上，可以将模型看作“更大的层”。Sequential类和Model类都是如此。这意味着可以在一个输入张量上调用模型，并得到一个输出张量。`y = model(x)`或`y1, y2 = model([x1, x2])`

通过重复使用模型实例可以构建一个简单的例子，就是一个使用双摄像头作为输入的视觉模型：两个平行的摄像头，相距几厘米（一英寸）。这样的模型可以感知深度，这在很多应用中
都很有用。不需要两个单独的模型从左右两个摄像头中分别提取视觉特征，然后再将二者合并。这样的底层处理可以在两个输入之间共享，即通过共享层（使用相同的权重，从而共享相同的
表示）来实现。在Keras 中实现连体视觉模型（共享卷积基）的代码如下所示。

In [ ]:
from keras import layers
from keras import applications
from keras import Input

# 图像处理基础模型是Xception网络（只包括卷积基）
xception_base = applications.Xception(weights=None,include_top=False)

# 输入是250×250 的RGB图像
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

# 对相同的视觉模型调用两次并合并特征
left_features = xception_base(left_input)
right_input = xception_base(right_input)
merged_features = layers.concatenate([left_features, right_input], axis=-1)